In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import os

In [97]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Num_CPUs = os.cpu_count()
print(f"Using {device} device")
print(f"Number of CPUs: {Num_CPUs}")

Using cuda device
Number of CPUs: 12


In [ ]:
data_transform = transforms.Compose([
    # Resize the images 
    transforms.Resize(size=(224, 224)),
    # Flip the images randomly 
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5), 
    # Turn the image into a torch.Tensor
    transforms.ToTensor()  
])

In [ ]:
TabletData = datasets.ImageFolder(root=r'c:\Users\Admin\OneDrive\Documents\Drexel\MEM679\datapath\TabletScans', transform=data_transform)
#print(TabletData)
#print(TabletData.classes)   
print(TabletData[0])

In [ ]:
import random
import matplotlib.pyplot as plt

# Get a random index
random_index = random.randint(0, len(TabletData) - 1)

# Get the image and label at the random index
image, label = TabletData[random_index]

# Get the class
class_name = TabletData.classes[label]

# Rearrange the order of dimensions
img_permute = image.permute(1, 2, 0)

# Plot the image
plt.figure(figsize=(10, 7))
plt.imshow(image.permute(1, 2, 0))
plt.axis("off")
plt.title(f"Class: {class_name}", fontsize=14);

In [ ]:
train_split, test_split = torch.utils.data.random_split(TabletData, [0.8, 0.2])
print(len(train_split), len(test_split))

# Only augment the training data
train_transforms = transforms.Compose([transforms.TrivialAugmentWide(num_magnitude_bins=31)])

train_dataloader = DataLoader(dataset=train_split, batch_size=64, num_workers=Num_CPUs, shuffle=True)
test_dataloader = DataLoader(dataset=test_split, batch_size=64, num_workers=Num_CPUs, shuffle=False) 

print(train_dataloader)
print(test_dataloader)


In [98]:
class ConvolutionalNN(nn.Module):
    """
    This is just TinyVGG.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1),  
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) 
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*16*16,
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

model0 = ConvolutionalNN(input_shape=3, # number of color channels (3 for RGB) 
                  hidden_units=10, 
                  output_shape=len(TabletData.classes)).to(device)
print(model0)


ConvolutionalNN(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2560, out_features=2, bias=True)
  )
)
